In [ ]:
# Import necessary libraries
import pandas as pd    # Data manipulation
import numpy as np     # Numerical operations
import glob            # File pattern matching
import matplotlib.pyplot as plt  # Plotting library

In [ ]:
#--- Data import ---
# Define a generic path to your CSV file (update this to your actual file location)
data_path = 'path/to/your/data.csv'
# Read the CSV into a DataFrame
data = pd.read_csv(data_path, delimiter=',')

In [ ]:
#--- Data exploration ---
# How many unique areas/polygons are in the dataset?
print('Number of areas/polygons:', data['ID'].nunique())

In [ ]:
#--- Date extraction and conversion ---
# Extract the date substring from the 'system:index' column and convert to datetime
data = data.assign(date=(data['system:index'].str[0:9]))
data['date'] = pd.to_datetime(data['date'])

data

In [ ]:
#--- Data filtering ---
# Filter out records with NDSI >= 0.4 to focus on clear conditions
filtered_data = data[data['NDSI'] < 0.4]

In [ ]:
#--- Split into per-ID tables ---
tables = []
# Loop over expected ID values, adjust upper bound as needed
for i in range(1, 214):
    table_name = f'K{i}'
    df_subset = filtered_data[filtered_data['ID'] == table_name].copy()
    if not df_subset.empty:
        tables.append(df_subset)

# Verify number of tables extracted
print('Extracted tables count:', len(tables))

#--- Re-parse dates for safety (if needed) ---
for i, df in enumerate(tables):
    # Check if 'system:index' exists in the DataFrame
    if 'system:index' in df.columns:
        try:
            # Extract the first 8 characters from the 'system:index' column and convert to datetime
            df.loc[:, 'date'] = pd.to_datetime(df['system:index'].str[:8], format='%Y%m%d')

            # Optionally, print the first few rows to verify
            print(f"Table {i+1} with Date Column:")
            print(df.head())
        
        except Exception as e:
            print(f"Error processing date for Table {i+1}: {e}")

    else:
        print(f"Table {i+1} does not contain 'system:index' column.")

NDVI CHARTS FOR EACH PHASES

In [ ]:
#--- Plotting NDVI time series by phase ---
# Define the phases to visualize (1 through 7)
phases_of_interest = range(1, 8)

# Configure subplot layout: 2 columns, automatic rows based on number of phases
ncols = 2
nrows = (len(phases_of_interest) + ncols - 1) // ncols
fig, axes = plt.subplots(nrows, ncols, figsize=(15, 5 * nrows), constrained_layout=True)
axes = axes.flatten()  # Flatten to easily iterate

# Loop through each phase and create a subplot
for i, phase in enumerate(phases_of_interest):
    all_series = []
    
    # Collect NDVI series for the current phase
    for df in tables:
        if isinstance(df, pd.DataFrame) and 'phase' in df.columns and 'NDVI' in df.columns:
            phase_data = df[df['phase'] == phase]
            if not phase_data.empty:
                all_series.append(phase_data[['date', 'NDVI']])

    # If there is data for this phase, plot it
    ax = axes[i]
    if all_series:
        all_data = pd.concat(all_series)
        mean_ndvi = all_data.groupby('date')['NDVI'].mean()

        # Plot all individual NDVI series in gray
        for series in all_series:
            ax.plot(series['date'], series['NDVI'], color='gray', alpha=0.5, linewidth=0.8)

        # Plot mean NDVI in color
        ax.plot(mean_ndvi.index, mean_ndvi.values, color='blue', linestyle='-', linewidth=2, label='Mean NDVI')

    # Labels and formatting
    ax.set_title(f'NDVI - Phase {phase}')
    ax.set_xlabel('Date')
    ax.set_ylabel('NDVI')
    ax.tick_params(axis='x', rotation=45)
    ax.set_ylim(0, 1)
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.6)

# Hide any unused subplots
for j in range(len(phases_of_interest), len(axes)):
    axes[j].axis('off')

# Save the figure to a file   
fig.savefig('NDVI_time_series.png', dpi=300, bbox_inches='tight')

# Display the plots
plt.show()